In [1]:
import requests, json
from pprint import pprint
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm
tqdm.pandas()

/Users/rachelchen/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

In [3]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    df['player_id'] = player_id
    
    return df

In [4]:
# create players dataframe
players = pd.json_normalize(r['elements'])
# create teams dataframe
teams = pd.json_normalize(r['teams'])
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

In [5]:
players.head(1)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,100.0,100.0,80201,0,0,-5,5,0,1,0.0,1.0,0,Bernd,0.0,1,False,,2022-02-11T08:00:15.144286Z,45,80201.jpg,1.3,Leno,0.8,False,None,a,1,3,4,64413,69,189597,103,0.0,0.9,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,367,29,600,69,581,60,425,30,NaN,,NaN,,NaN,


In [6]:
teams.head(1)

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1


In [7]:
positions.head(1)

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83


In [17]:
# join team name
df = players.merge(
    teams,
    left_on='team',
    right_on='id',
    suffixes=['_player', '_team']
).drop(['team', 'id_team'], axis=1)

# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id',
    suffixes=[None, '_pos']
).drop(['element_type', 'id'], axis=1)

df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code_player,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,ep_next,ep_this,event_points,first_name,form_player,id_player,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,code_team,draw,form_team,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,100.0,100.0,80201,0,0,-5,5,0,0.0,1.0,0,Bernd,0.0,1,False,,2022-02-11T08:00:15.144286Z,45,80201.jpg,1.3,Leno,0.8,False,None,a,3,4,64413,69,189597,103,0.0,0.9,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,367,29,600,69,581,60,425,30,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
1,0.0,0.0,115918,0,0,0,0,0,0.0,0.0,0,Rúnar Alex,0.0,2,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,115918.jpg,0.0,Rúnarsson,0.6,False,None,u,3,0,19017,0,78367,71,0.0,0.0,Rúnarsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,523,53,509,32,471,19,527,53,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
2,0.0,0.0,463748,0,0,0,0,0,0.0,0.0,0,Karl,0.0,532,False,Joined Reading on loan until the end of the se...,2022-01-24T22:30:12.592957Z,40,463748.jpg,0.0,Hein,0.4,False,None,u,3,0,50944,0,40819,135,0.0,0.0,Hein,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,555,66,541,45,504,32,559,66,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
3,NaN,NaN,225321,0,0,6,-6,3,7.6,4.8,2,Aaron,3.8,559,False,,None,51,225321.jpg,4.9,Ramsdale,20.3,False,None,a,3,107,2638892,10258,990808,6335,0.7,21.0,Ramsdale,1980,0,0,11,20,0,0,0,1,0,67,12,472,493.4,1.0,0.0,49.4,55,13,454,11,688,75,243,13,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
4,NaN,NaN,220682,0,0,-1,1,0,0.0,1.0,0,Arthur,0.0,572,False,,None,39,220682.jpg,0.0,Okonkwo,0.6,False,None,a,3,0,68840,1991,24136,568,0.0,0.0,Okonkwo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,509,46,492,22,453,9,513,46,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83


In [10]:
# get gameweek histories for each player
points = df['id_player'].progress_apply(get_season_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

In [19]:
# join web_name
df = df.merge(
    points,
    left_on='id_player',
    right_on='player_id',
    suffixes=[None, '_pts']
).drop(['player_id'], axis=1)

In [20]:
df21_detail = df[df['season_name']=='2020/21']
df21_detail.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code_player,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,ep_next,ep_this,event_points,first_name,form_player,id_player,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,code_team,draw,form_team,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count,season_name,element_code,start_cost,end_cost,total_points_pts,minutes_pts,goals_scored_pts,assists_pts,clean_sheets_pts,goals_conceded_pts,own_goals_pts,penalties_saved_pts,penalties_missed_pts,yellow_cards_pts,red_cards_pts,saves_pts,bonus_pts,bps_pts,influence_pts,creativity_pts,threat_pts,ict_index_pts
2,100.0,100.0,80201,0,0,-5,5,0,0.0,1.0,0,Bernd,0.0,1,False,,2022-02-11T08:00:15.144286Z,45,80201.jpg,1.3,Leno,0.8,False,None,a,3,4,64413,69,189597,103,0.0,0.9,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,367,29,600,69,581,60,425,30,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83,2020/21,80201.0,50.0,50.0,131.0,3131.0,0.0,0.0,11.0,37.0,1.0,1.0,0.0,0.0,1.0,86.0,11.0,625.0,702.2,0.0,2.0,70.3
3,0.0,0.0,115918,0,0,0,0,0,0.0,0.0,0,Rúnar Alex,0.0,2,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,115918.jpg,0.0,Rúnarsson,0.6,False,None,u,3,0,19017,0,78367,71,0.0,0.0,Rúnarsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,523,53,509,32,471,19,527,53,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83,2020/21,115918.0,45.0,44.0,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8.0,16.6,0.0,0.0,1.7
7,NaN,NaN,225321,0,0,6,-6,3,7.6,4.8,2,Aaron,3.8,559,False,,None,51,225321.jpg,4.9,Ramsdale,20.3,False,None,a,3,107,2638892,10258,990808,6335,0.7,21.0,Ramsdale,1980,0,0,11,20,0,0,0,1,0,67,12,472,493.4,1.0,0.0,49.4,55,13,454,11,688,75,243,13,NaN,,NaN,,NaN,,3,0,None,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83,2020/21,225321.0,50.0,46.0,123.0,3420.0,0.0,0.0,5.0,63.0,0.0,0.0,0.0,1.0,0.0,146.0,12.0,685.0,1023.0,20.0,4.0,104.7
13,0.0,0.0,79852,0,0,-1,1,0,0.0,0.0,0,Jed,0.0,28,False,Joined Luton Town on loan until the end of the...,2022-01-31T18:00:10.386198Z,39,79852.jpg,1.0,Steer,2.1,False,None,u,7,1,325469,0,264553,913,0.0,0.3,Steer,90,0,0,0,3,0,0,0,0,0,1,0,13,8.2,0.0,0.0,0.8,466,38,487,18,448,5,483,38,NaN,,NaN,,NaN,,7,0,None,0,Aston Villa,0,0,0,AVL,3,None,False,0,1130,1160,1140,1110,1090,1090,2,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83,2020/21,79852.0,40.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,100.0,100.0,98980,0,0,0,0,1,2.5,6.0,6,Emiliano,3.0,30,False,,2021-10-13T23:00:12.628419Z,55,98980.jpg,3.7,Martínez,12.1,False,None,a,7,93,642257,1003,2220766,3301,0.5,16.9,Martínez,2250,0,0,8,34,0,1,0,4,0,66,7,457,565.2,0.0,0.0,56.3,27,9,700,81,699,80,220,10,NaN,,NaN,,NaN,,7,0,None,0,Aston Villa,0,0,

In [27]:
df21_project = df21_detail[['id_player','first_name', 'second_name', 'name', 'singular_name', 
                            'end_cost', 'total_points_pts']
                          ].sort_values(by=['total_points_pts'], ascending = False)
df21_project.head(10)

,id_player,first_name,second_name,name,singular_name,end_cost,total_points_pts
883,277,Bruno Miguel,Borges Fernandes,Man Utd,Midfielder,113.0,244.0
1433,357,Harry,Kane,Spurs,Forward,119.0,242.0
774,233,Mohamed,Salah,Liverpool,Midfielder,129.0,231.0
1038,359,Heung-Min,Son,Spurs,Midfielder,96.0,228.0
1340,189,Patrick,Bamford,Leeds,Forward,66.0,194.0
1329,205,Jamie,Vardy,Leicester,Forward,102.0,187.0
20,30,Emiliano,Martínez,Aston Villa,Goalkeeper,53.0,186.0
765,230,Sadio,Mané,Liverpool,Midfielder,118.0,176.0
889,281,Marcus,Rashford,Man Utd,Midfielder,96.0,174.0
690,188,Stuart,Dallas,Leeds,Midfielder,55.0,171.0


In [30]:
df21_detail.to_csv('df21_detail.csv', index = False)
df21_project.to_csv('df21_project.csv', index = False)